In [ ]:
!pip install torch transformers accelerator datasets evaluate tqdm pandas nltk scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour 

## Import Aspect Classification Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("ilos-vigil/steam-review-aspect-dataset")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.68k [00:00<?, ?B/s]

data-00000-of-00001.arrow:   0%|          | 0.00/2.59M [00:00<?, ?B/s]

data-00000-of-00001.arrow:   0%|          | 0.00/529k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/900 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['appid', 'review', 'cleaned_review', 'labels'],
        num_rows: 900
    })
    test: Dataset({
        features: ['appid', 'review', 'cleaned_review', 'labels'],
        num_rows: 200
    })
})

## Load Aspect Classification Model

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

aspect_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
aspect_model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 8
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Preprocess Text 1

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

#!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    tokens = [word for word in tokens if word.lower() not in stop_words]

    return ' '.join(tokens)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


## Fine-tune Model

In [ ]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, hamming_loss
import torch
from nltk.tokenize import sent_tokenize
import os

os.environ["WANDB_DISABLED"] = "true"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
aspect_model = aspect_model.to(device)

def preprocess_data(examples):
    examples["review"] = [preprocess_text(review) for review in examples["review"]]
    inputs = aspect_tokenizer(examples["review"], truncation=True, padding="max_length", max_length=128)
    inputs["labels"] = examples["labels"]  # Dataset has multi-labels for each review
    return inputs

tokenized_datasets = dataset.map(preprocess_data, batched=True)

def compute_metrics(pred):
    logits, labels = pred
    predictions = (logits > 0.5).astype(int)

    # Compute metrics
    precision = precision_score(labels, predictions, average="micro")
    recall = recall_score(labels, predictions, average="micro")
    f1 = f1_score(labels, predictions, average="micro")

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_steps=10,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10
)

trainer = Trainer(
    model=aspect_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=aspect_tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-5-ca2eeab2df5b>:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.558700,0.552651,0.770619,0.457187,0.573896
2,0.510100,0.511524,0.776786,0.532110,0.631579
3,0.476300,0.498622,0.742525,0.683486,0.711783
4,0.433300,0.456842,0.834423,0.585627,0.688230
5,0.401100,0.442212,0.831094,0.662080,0.737021
6,0.352800,0.438300,0.825336,0.657492,0.731915
7,0.359100,0.430849,0.825581,0.651376,0.728205
8,0.335700,0.430580,0.842105,0.636086,0.724739
9,0.298100,0.426707,0.830739,0.652905,0.731164
10,0.313100,0.425018,0.829868,0.671254,0.742181


TrainOutput(global_step=570, training_loss=0.41040321651257966, metrics={'train_runtime': 690.1037, 'train_samples_per_second': 13.042, 'train_steps_per_second': 0.826, 'total_flos': 592031766528000.0, 'train_loss': 0.41040321651257966, 'epoch': 10.0})

## Evaluate Aspect Classification Model

In [ ]:
import torch
import numpy as np

# Define aspects and threshold
aspect_labels = ["Recommended", "Story", "Gameplay", "Visual", "Audio", "Technical", "Price", "Suggestions"]
threshold = 0.6

# Prepare ground truth and predictions
ground_truth = []
predictions = []

# Iterate through the dataset to evaluate
for example in dataset["test"]:  # Replace "test" with the relevant split
    # Ground truth for this sample
    ground_truth.append(example["labels"])  # Assuming multi-hot encoding for ground truth labels

    # Model prediction for this sample
    example["review"] = preprocess_text(example["review"])
    inputs = aspect_tokenizer(example["review"], return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = aspect_model(**inputs)
    probs = torch.sigmoid(outputs.logits).squeeze().cpu().numpy()

    # Convert probabilities to binary predictions
    binary_predictions = (probs > threshold).astype(int)
    predictions.append(binary_predictions)

# Convert to numpy arrays for evaluation
ground_truth = np.array(ground_truth)
predictions = np.array(predictions)

# Compute metrics
precision = precision_score(ground_truth, predictions, average="micro")
recall = recall_score(ground_truth, predictions, average="micro")
f1 = f1_score(ground_truth, predictions, average="micro")
hamming = hamming_loss(ground_truth, predictions)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"Hamming Loss: {hamming:.4f}")

Precision: 0.8305
Recall: 0.7416
F1-Score: 0.7835
Hamming Loss: 0.1675


## Aspect Extraction

In [ ]:
review = """The best game ever made by Valve. Great story, beautiful dialogues with funny jokes, interesting puzzles, perfect atmosphere and a lot of fun in co-op with friends. You made a great work, Valve, and thank You for this absolute masterpiece. 10/10."""
sentences = sent_tokenize(review)

aspect_labels = ["Recommended", "Story", "Gameplay", "Visual", "Audio", "Technical", "Price", "Suggestions"]
aspect_sentences = {aspect: [] for aspect in aspect_labels}

for sentence in sentences:
    inputs = aspect_tokenizer(sentence, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = aspect_model(**inputs)

    probs = torch.sigmoid(outputs.logits).squeeze().cpu().numpy()
    for idx, prob in enumerate(probs):
        if prob > 0.6:  # Threshold for relevance
            aspect_sentences[aspect_labels[idx]].append((sentence, prob))

print(aspect_sentences)

{'Recommended': [('The best game ever made by Valve.', 0.7121258), ('Great story, beautiful dialogues with funny jokes, interesting puzzles, perfect atmosphere and a lot of fun in co-op with friends.', 0.9417423), ('You made a great work, Valve, and thank You for this absolute masterpiece.', 0.6435783), ('10/10.', 0.7524089)], 'Story': [('Great story, beautiful dialogues with funny jokes, interesting puzzles, perfect atmosphere and a lot of fun in co-op with friends.', 0.821667)], 'Gameplay': [('Great story, beautiful dialogues with funny jokes, interesting puzzles, perfect atmosphere and a lot of fun in co-op with friends.', 0.7304769)], 'Visual': [], 'Audio': [], 'Technical': [], 'Price': [], 'Suggestions': []}


## Formatting Output

In [ ]:
def concat_aspect_sentences(aspect_sentences):
    new_dict = {}
    for aspect in aspect_sentences:
        if(aspect_sentences[aspect] != []):
            text = ""
            for tuple in aspect_sentences[aspect]:
                text = text + " " + tuple[0]
            new_dict[aspect] = text

    return new_dict

## Import Sentiment Classification Dataset

In [ ]:
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split

# review_data = pd.read_csv("/content/reviews.csv")
# review_df = pd.DataFrame(review_data)

tweet_data = load_dataset("SetFit/tweet_sentiment_extraction")
tweet_df_train = pd.DataFrame(tweet_data['train'])[:800]
tweet_df_test = pd.DataFrame(tweet_data['test'])[:200]

tweet_data

README.md:   0%|          | 0.00/94.0 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl:   0%|          | 0.00/3.93M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/503k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/27481 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3534 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['textID', 'text', 'label', 'label_text'],
        num_rows: 27481
    })
    test: Dataset({
        features: ['textID', 'text', 'label', 'label_text'],
        num_rows: 3534
    })
})

## Reshape Data

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def score_to_sentiment(score):
    if score >= 60:
        return 2
    elif score >= 40:
        return 1
    else:
        return 0

def preprocess_data(text):
    text = lemmatizer.lemmatize(text.lower())
    return text.lower()

# Drop irrelevant columns
tweet_df_train = tweet_df_train.drop("textID", axis=1)
tweet_df_test = tweet_df_test.drop("textID", axis=1)
tweet_df_train = tweet_df_train.drop("label_text", axis=1)
tweet_df_test = tweet_df_test.drop("label_text", axis=1)
# review_df = review_df.drop('Website', axis=1)
# review_df = review_df.drop('ID', axis=1)
# review_df = review_df.drop('Game', axis=1)
# review_df.dropna(inplace=True)

# Rename columns
# review_df.rename(columns={"Score": "label"}, inplace=True)
# review_df.rename(columns={"Review": "text"}, inplace=True)

# Change values
# review_df["label"] = review_df["label"].apply(score_to_sentiment)
# review_df["text"] = review_df["text"].apply(preprocess_data)

# positive_df = review_df[review_df['label'] == 2].sample(frac=1)[:500]
# neutral_df = review_df[review_df['label'] == 1].sample(frac=1)[:400]
# negative_df = review_df[review_df['label'] == 0].sample(frac=1)[:350]

# review_df = pd.concat([positive_df, neutral_df, negative_df], axis=0)

tweet_df_train["text"] = tweet_df_train["text"].apply(preprocess_data)
tweet_df_test["text"] = tweet_df_test["text"].apply(preprocess_data)

# Save modified
# review_df.to_csv("modified_reviews.csv", index=False)
# tweet_df_train.to_csv("modified_tweets_train.csv", index=False)
# tweet_df_test.to_csv("modified_tweets_test.csv", index=False)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Initiate Sentiment Classification Model and Tokenizer

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

sentiment_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
sentiment_model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 3
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoTokenizer, BertTokenizer
from datasets import Dataset

# Tokenize the dataset
def tokenize_data(examples):
    return sentiment_tokenizer(examples["text"], padding="max_length")

# review_df_train, review_df_test = train_test_split(review_df, test_size=0.2, random_state=42)
# review_dataset_train = Dataset.from_pandas(review_df_train)
# review_dataset_test = Dataset.from_pandas(review_df_test)
tweet_dataset_train = Dataset.from_pandas(tweet_df_train)
tweet_dataset_test = Dataset.from_pandas(tweet_df_test)

# tokenized_review_train = review_dataset_train.map(tokenize_data, batched=True)
# tokenized_review_test = review_dataset_test.map(tokenize_data, batched=True)
tokenized_tweet_train = tweet_dataset_train.map(tokenize_data, batched=True)
tokenized_tweet_test = tweet_dataset_test.map(tokenize_data, batched=True)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

## Fine-tune Model for Sentiment Classification

In [ ]:
from transformers import Trainer, TrainingArguments
import torch
from nltk.tokenize import sent_tokenize
import os

os.environ["WANDB_DISABLED"] = "true"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sentiment_model = sentiment_model.to(device)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_steps=10,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, hamming_loss

def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=1)

    # Compute metrics
    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average="weighted")
    recall = recall_score(labels, predictions, average="weighted")
    f1 = f1_score(labels, predictions, average="weighted")

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

trainer = Trainer(
    model=sentiment_model,
    args=training_args,
    train_dataset=tokenized_tweet_train,
    eval_dataset=tokenized_tweet_test,
    compute_metrics=compute_metrics,
    tokenizer=sentiment_tokenizer,
)

trainer.train()

<ipython-input-14-7defe267ae82>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.063200,1.037321,0.510000,0.597238,0.510000,0.434993
2,0.898400,0.902824,0.630000,0.662506,0.630000,0.631170
3,0.639500,0.692929,0.740000,0.743695,0.740000,0.740050
4,0.595000,0.630120,0.710000,0.730013,0.710000,0.710822
5,0.398900,0.540579,0.780000,0.781681,0.780000,0.780584
6,0.364700,0.580061,0.755000,0.766664,0.755000,0.753260
7,0.237600,0.567391,0.755000,0.760715,0.755000,0.754392
8,0.191500,0.575622,0.750000,0.755815,0.750000,0.748811
9,0.167600,0.581720,0.750000,0.755905,0.750000,0.748023
10,0.114300,0.573379,0.765000,0.769359,0.765000,0.765182


TrainOutput(global_step=500, training_loss=0.4836223440170288, metrics={'train_runtime': 1134.0577, 'train_samples_per_second': 7.054, 'train_steps_per_second': 0.441, 'total_flos': 2104907341824000.0, 'train_loss': 0.4836223440170288, 'epoch': 10.0})

## Evaluate Sentiment Classification Model

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, hamming_loss

# Prepare ground truth and predictions
ground_truth = []
predictions = []

# Iterate through the dataset to evaluate
for index, row in tweet_df_test.iterrows():  # Replace "test" with the relevant split
    # Ground truth for this sample
    ground_truth.append(row["label"])  # Assuming multi-hot encoding for ground truth labels

    # Model prediction for this sample
    inputs = sentiment_tokenizer(row["text"], return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = sentiment_model(**inputs)
    # Extract logits and compute predicted class
    logits = outputs.logits  # Assuming your model outputs a `logits` tensor
    predicted_class = torch.argmax(logits, dim=-1).item()  # Get the class index with the highest score

    # Append prediction
    predictions.append(predicted_class)

# Convert to numpy arrays for evaluation
ground_truth = np.array(ground_truth)
predictions = np.array(predictions)
print

# Compute metrics
accuracy = accuracy_score(ground_truth, predictions)
precision = precision_score(ground_truth, predictions, average="weighted")
recall = recall_score(ground_truth, predictions, average="weighted")
f1 = f1_score(ground_truth, predictions, average="weighted")
hamming = hamming_loss(ground_truth, predictions)

# Output metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Hamming Loss: {hamming}")

Accuracy: 0.765
Precision: 0.7693588093743461
Recall: 0.765
F1 Score: 0.7651819118427566
Hamming Loss: 0.235


In [ ]:
review = "It wants to be so many games, and for each of its inspirations, it delivers a different demand. As a logical and unfortunate result, Pokemon Conquest is just too damned demanding."
review = "Unfortunately Flatout 3 is a big disapointment for me and many fans of previous parts.What went wrong?Gameplay has lost this \"catchy\" spirit making this game addicting. Controls has changed and physics as well. You don't have any feeling of the vehicle You driving. The game seems to be at faster pace. Too fast form me :) Too many explosions and destructions out of a good taste. Soundtrack is flat and colorless. It isn't bad but... There is no tracks of some famous rockbands as previously. Graphisc has not made any progress. It has made a step backwards as it stay in place. Flatout3 has a lot of game modes some of which are not playable at all. I like SPEED Mode it has something good :) The rest is kind of fun ... for about 5 minutes. There is no career mode :( To be honest : Flatout 1 & 2 have their own style and leave in gamers' minds an unique impresion and lots of good, warm memories. FO3 does not..."

inputs = sentiment_tokenizer(review, return_tensors="pt").to(device)
with torch.no_grad():
    outputs = sentiment_model(**inputs)

logits = outputs.logits  # Assuming your model outputs a `logits` tensor
predicted_class = torch.argmax(logits, dim=-1).item()  # Get the class index with the highest score

if predicted_class == 2:
  print("Positive")
elif predicted_class == 1:
  print("Neutral")
else:
  print("Negative")

Neutral


## Test Full Pipeline

In [ ]:
review = """The best game ever made by Valve. Great story, beautiful dialogues with funny jokes, interesting puzzles, perfect atmosphere and a lot of fun in co-op with friends. You made a great work, Valve, and thank You for this absolute masterpiece. 10/10."""
sentences = sent_tokenize(review)

aspect_labels = ["Recommended", "Story", "Gameplay", "Visual", "Audio", "Technical", "Price", "Suggestions"]
aspect_sentences = {aspect: [] for aspect in aspect_labels}

for sentence in sentences:
    inputs = aspect_tokenizer(sentence, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = aspect_model(**inputs)

    probs = torch.sigmoid(outputs.logits).squeeze().cpu().numpy()
    for idx, prob in enumerate(probs):
        if prob > 0.6:  # Threshold for relevance
            aspect_sentences[aspect_labels[idx]].append((sentence, prob))

review_aspect = concat_aspect_sentences(aspect_sentences)

for aspect in review_aspect:
    aspect_text = review_aspect[aspect]
    inputs = sentiment_tokenizer(aspect_text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = sentiment_model(**inputs)

    logits = outputs.logits  # Assuming your model outputs a `logits` tensor
    predicted_class = torch.argmax(logits, dim=-1).item()  # Get the class index with the highest score

    print(aspect, ":", aspect_text)
    if predicted_class == 2:
      print("Positive")
    elif predicted_class == 1:
      print("Neutral")
    else:
      print("Negative")

Recommended :  The best game ever made by Valve. Great story, beautiful dialogues with funny jokes, interesting puzzles, perfect atmosphere and a lot of fun in co-op with friends. You made a great work, Valve, and thank You for this absolute masterpiece. 10/10.
Positive
Story :  Great story, beautiful dialogues with funny jokes, interesting puzzles, perfect atmosphere and a lot of fun in co-op with friends.
Positive
Gameplay :  Great story, beautiful dialogues with funny jokes, interesting puzzles, perfect atmosphere and a lot of fun in co-op with friends.
Positive


## Train from Tweet Dataset

In [ ]:
# Define the directory to save the model
save_directory = "/content/drive/MyDrive/NLP"

# Save the model
sentiment_model.save_pretrained(save_directory)

# If using a tokenizer, save it as well
sentiment_tokenizer.save_pretrained(save_directory)

('/content/drive/MyDrive/NLP/tokenizer_config.json',
 '/content/drive/MyDrive/NLP/special_tokens_map.json',
 '/content/drive/MyDrive/NLP/vocab.txt',
 '/content/drive/MyDrive/NLP/added_tokens.json')

In [ ]:
# from transformers import pipeline

# classifier = pipeline("text-classification", model="./sentiment_model", tokenizer=tokenizer)
# print(classifier("The game is fantastic!"))
